In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
import random
warnings.filterwarnings('ignore')

In [2]:
import matplotlib.font_manager as fm

[(f.name, f.fname) for f in fm.fontManager.ttflist if 'Nanum' in f.name]
plt.rcParams['font.family'] = 'Malgun Gothic'

In [3]:
df_01 = pd.read_csv('../bigdata/LPOINT_BIG_COMP/LPOINT_BIG_COMP_01_DEMO.csv')
df_02 = pd.read_csv('../bigdata/LPOINT_BIG_COMP/LPOINT_BIG_COMP_02_PDDE.csv')
df_03 = pd.read_csv('../bigdata/LPOINT_BIG_COMP/LPOINT_BIG_COMP_03_COP_U.csv')
df_04 = pd.read_csv('../bigdata/LPOINT_BIG_COMP/LPOINT_BIG_COMP_04_PD_CLAC.csv')
df_05 = pd.read_csv('../bigdata/LPOINT_BIG_COMP/LPOINT_BIG_COMP_05_BR.csv')
df_06 = pd.read_csv('../bigdata/LPOINT_BIG_COMP/LPOINT_BIG_COMP_06_LPAY.csv')

In [4]:
df_02 = df_02[df_02['chnl_dv'] == 1]

In [5]:
df_02.drop(columns=['buy_am', 'de_dt', 'rct_no', 'chnl_dv'], axis=1, inplace=True)

In [6]:
df_merge_02_04 = pd.merge(df_02, df_04, on='pd_c')
df_merge_02_04 

,cust,cop_c,br_c,pd_c,de_hr,buy_ct,pd_nm,clac_hlv_nm,clac_mcls_nm
0,M430112881,A01,A010039,PD0290,10,1,남성티셔츠,남성의류,남성의류상의
1,M430112881,A01,A010039,PD0290,10,1,남성티셔츠,남성의류,남성의류상의
2,M430112881,A01,A010039,PD0290,10,1,남성티셔츠,남성의류,남성의류상의
3,M430112881,A01,A010039,PD0290,10,1,남성티셔츠,남성의류,남성의류상의
4,M506355556,A01,A010032,PD0290,15,1,남성티셔츠,남성의류,남성의류상의
...,...,...,...,...,...,...,...,...,...
3953237,M033482606,A05,A050003,PD0075,20,1,혈당계/혈압계,건강용품,건강측정용품
3953238,M489266563,A05,A050426,PD1221,15,1,청소기/광택기,자동차용품,세차용품
3953239,M296090374,A05,A050260,PD0543,11,1,이발기,생활/주방가전,이미용가전
3953240,M560002381,A05,A050039,PD0543,12,1,이발기,생활/주방가전,이미용가전


### 상품 상위 5개 추출

In [7]:
def frequency_product(cust, df_merge_02_04):
    person_id = cust
    pd_c = df_merge_02_04[df_merge_02_04['cust'] == person_id][['clac_hlv_nm', 'buy_ct']]
    pd_c = pd_c.groupby('clac_hlv_nm')['buy_ct'].sum().reset_index()
    pd_c = pd_c.sort_values(by='buy_ct', ascending=False, ignore_index=True)
    freq_product = []
    for i in range(5):
        try:
            freq_product.append(pd_c['clac_hlv_nm'][i])

        except: KeyError
            
    return freq_product

In [8]:
frequency_product('M430112881', df_merge_02_04)

['채소', '냉장식품', '수산물', '유제품', '음료']

In [ ]:
df_01['freq_product'] = df_01.apply(lambda x : frequency_product(x['cust'], df_merge_02_04), axis=1)

### 평점 변환

In [10]:
def convert_rating(cust, clac_hlv_nm, df):
    
    person_id = cust    
    try:
        pd_c = df[df['cust'] == cust][['clac_hlv_nm', 'buy_ct']]
        pd_c = pd_c.groupby('clac_hlv_nm')['buy_ct'].sum().reset_index()
        pd_c = pd_c.sort_values(by='buy_ct', ascending=False, ignore_index=True)

        pd_c['buy_ct'] = (pd_c['buy_ct'] - pd_c['buy_ct'].min()) / (pd_c['buy_ct'].max() - pd_c['buy_ct'].min())
        pd_c['buy_ct'][0] = random.uniform(0.85, 1.01) 
        pd_c['buy_ct'] = np.round(pd_c['buy_ct'], 2) * 10

        res = float(pd_c[pd_c['clac_hlv_nm'] == clac_hlv_nm]['buy_ct'])
        
    except: ValueError
        
    return res

In [11]:
res = convert_rating(df_merge_02_04['cust'][10], df_merge_02_04['clac_hlv_nm'][10], df_merge_02_04)
res

3.8

In [ ]:
df_merge_02_04['ratings'] = df_merge_02_04.apply(lambda x : convert_rating(x['cust'], x['clac_hlv_nm'], df_merge_02_04), axis=1)

### 멀티프로세싱

In [318]:
import multiprocessing as mp

num_cores = 6

def parallel_dataframe(df, func):
    global num_cores
    df_split = np.array_split(df, num_cores)
    pool = mp.Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    
    return df

In [319]:
def make_ratings(df_merge_02_04):
    df_merge_02_04['ratings'] = df_merge_02_04[:50].apply(lambda x : convert_rating(x['cust'], x['clac_hlv_nm'], df_merge_02_04), axis=1)
    return df_merge_02_04

In [ ]:
parallel_dataframe(df_merge_02_04, make_ratings)

In [281]:
df_merge_02_04

,cust,cop_c,br_c,pd_c,de_hr,buy_ct,pd_nm,clac_hlv_nm,clac_mcls_nm,ratings
0,M430112881,A01,A010039,PD0290,10,1,남성티셔츠,남성의류,남성의류상의,1.0
1,M430112881,A01,A010039,PD0290,10,1,남성티셔츠,남성의류,남성의류상의,1.0
2,M430112881,A01,A010039,PD0290,10,1,남성티셔츠,남성의류,남성의류상의,1.0
3,M430112881,A01,A010039,PD0290,10,1,남성티셔츠,남성의류,남성의류상의,1.0
4,M506355556,A01,A010032,PD0290,15,1,남성티셔츠,남성의류,남성의류상의,3.8
...,...,...,...,...,...,...,...,...,...,...
3953237,M033482606,A05,A050003,PD0075,20,1,혈당계/혈압계,건강용품,건강측정용품,NaN
3953238,M489266563,A05,A050426,PD1221,15,1,청소기/광택기,자동차용품,세차용품,NaN
3953239,M296090374,A05,A050260,PD0543,11,1,이발기,생활/주방가전,이미용가전,NaN
3953240,M560002381,A05,A050039,PD0543,12,1,이발기,생활/주방가전,이미용가전,NaN


In [254]:
cust = df_merge_02_04['cust'][20]
pd_c = df_merge_02_04[df_merge_02_04['cust'] == cust][['clac_hlv_nm', 'buy_ct']]
pd_c = pd_c.groupby('clac_hlv_nm')['buy_ct'].sum().reset_index()
pd_c = pd_c.sort_values(by='buy_ct', ascending=False, ignore_index=True)

pd_c['buy_ct'] = (pd_c['buy_ct'] - pd_c['buy_ct'].min()) / (pd_c['buy_ct'].max() - pd_c['buy_ct'].min())
pd_c['buy_ct'][0] = random.uniform(0.85, 1.01) 
pd_c['buy_ct'] = np.round(pd_c['buy_ct'], 2) * 10
pd_c

# freq_product = []
# for i in range(5):
#     try:
#         freq_product.append(pd_c['clac_hlv_nm'][i])
    
#     except: KeyError
    
# freq_product

,clac_hlv_nm,buy_ct
0,주류,9.2
1,채소,8.8
2,퍼스널케어,6.2
3,음료,6.2
4,여성의류,5.0
5,수산물,2.5
6,유제품,2.5
7,과일,2.5
8,대용식,2.5
9,식기/조리기구,1.2


In [186]:
cust = df_01['cust'][30]
pd_c = df_merge_02_04[df_merge_02_04['cust'] == cust][['clac_hlv_nm', 'buy_ct']]
pd_c = pd_c.groupby('clac_hlv_nm').size().reset_index()
pd_c = pd_c.sort_values(by=0, ascending=False, ignore_index=True)
pd_c

,clac_hlv_nm,0
0,테넌트/음식점,25
1,대용식,15
2,커피/차,12
3,수산물,5
4,채소,4
5,과자,4
6,유제품,4
7,주류,4
8,축산물,3
9,과일,3


In [178]:
cust = df_01['cust'][20]
pd_c = df_merge_02_04[df_merge_02_04['cust'] == cust]['clac_hlv_nm']
pd_c = pd_c.value_counts(normalize=True).reset_index()
pd_c.columns = ['clac_hlv_nm', 'frequency']

prod = {}
i = 0
for p, v in zip(pd_c['clac_hlv_nm'], pd_c['frequency']):
    prod[i] = {f'{p}' : f'{round(v, 3)}'}
    i += 1

In [99]:
prod

{0: {'여성의류': '0.177'},
 1: {'남성의류': '0.156'},
 2: {'패션잡화': '0.094'},
 3: {'속옷/양말/홈웨어': '0.094'},
 4: {'침구/수예': '0.083'},
 5: {'아웃도어/레저': '0.042'},
 6: {'식기/조리기구': '0.031'},
 7: {'헬스/피트니스': '0.031'},
 8: {'문구/사무용품': '0.031'},
 9: {'대용식': '0.021'},
 10: {'테넌트/음식점': '0.021'},
 11: {'유아동의류': '0.021'},
 12: {'화장품/뷰티케어': '0.021'},
 13: {'냉동식품': '0.021'},
 14: {'음료': '0.021'},
 15: {'과일': '0.021'},
 16: {'스포츠패션': '0.01'},
 17: {'모바일': '0.01'},
 18: {'주류': '0.01'},
 19: {'생활/주방가전': '0.01'},
 20: {'퍼스널케어': '0.01'},
 21: {'조리식품': '0.01'},
 22: {'병통조림': '0.01'},
 23: {'세제/위생': '0.01'},
 24: {'조미료': '0.01'},
 25: {'양곡': '0.01'},
 26: {'채소': '0.01'}}

In [39]:
df_04[df_04['pd_c'] == 'PD1152']

,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm
1151,PD1152,가공우유,유제품,가공유
